## Name: FileCompiler

In [1]:
#List necessary packages
import glob as gl
import pandas as pd
import numpy as np
import os
import re

### Name: MSCompiler

#### What It Does: 
Extracts desired column/row values from all brand .csv mainsheet files.

#### How To Use It: 
-

#### How It Works:
Indexes all brand folder, 
opens each folder, 
identifies files with "main--" prefix, 
opens them, 
subsets them as desired, 
joins appends data to "CompiledMainSheet" dataframe, 
exports "CompiledMainSheet" to specified location.

#### Troubleshooting & Notes:
-

In [ ]:
#Check id duplicates of main--*.csv files and resolve any issues

def MScheck():

    AllBrandFolderPath = "//192.168.2.32/GoogleDrive/Completed Magento Uploads (v 1.0)"
    AllBrandFolders = os.listdir(AllBrandFolderPath)

    for b in range(len(AllBrandFolders)):

        BrandPath = AllBrandFolderPath+"/"+AllBrandFolders[b]
        BrandMSpath = BrandPath+"/main--*.csv"
        BrandSSpath = BrandPath+"/sub--*.csv"
        MSfilecount = len(gl.glob(BrandMSpath))
        SSfilecount = len(gl.glob(BrandSSpath))
        if MSfilecount != 1:
            print(AllBrandFolders[b], "MS", MSfilecount)
        #if SSfilecount != 1:
            #print(AllBrandFolders[b], "SS", SSfilecount)

MScheck()

In [ ]:
#Compile all brand mainsheet data into "CompiledMainSheet" dataframe

def MScompile():
    
    low_memory=False
    CompiledMainSheet = pd.DataFrame
    AllColumns = []
    
    AllBrandFolderPath = "//192.168.2.32/GoogleDrive/Completed Magento Uploads (v 1.0)"
    AllBrandFolders = os.listdir(AllBrandFolderPath)

    for b in range(len(AllBrandFolders)):
        print((len(AllBrandFolders)-b))
        BrandPath= str(AllBrandFolderPath+"/"+AllBrandFolders[b])
        BrandMSpath = BrandPath+"/main--*.csv"
        filecount = len(gl.glob(BrandMSpath))
        if filecount == 1:
            MSFile = gl.glob(BrandMSpath)[0]     
            BrandMS = list(map(str.strip, (pd.read_csv(MSFile, encoding='mac_roman').columns)))
            AllColumns = list(set(AllColumns + BrandMS))
    return(AllColumns)


              
len(AllColumns)
cols = MScompile()
b = 1

AllColumns.strip()

type(AllColumns)
CompiledMainSheet = CompiledMainSheet.append(BrandMS)


In [ ]:
#Pull out tire and wheel filters from all subsheets

WheelDataAll = pd.DataFrame(columns = ["internal_sku","wheel_diameter_pd","wheel_width_pd","bolt_pattern_pd","offset_pd","center_bore_pd", "brand"])
TireDataAll = pd.DataFrame(columns = ["internal_sku", "tire_width_pd", "tire_profile_pd", "tire_wheel_dia_pd", "brand"])
    
    
AllBrandFolderPath = "//192.168.2.32/GoogleDrive/Completed Magento Uploads (v 1.0)"
AllBrandFolders = os.listdir(AllBrandFolderPath)

for b in range(len(AllBrandFolders)):
    print((len(AllBrandFolders)-b))
    Brand = AllBrandFolders[b]
    BrandPath= str(AllBrandFolderPath+"/"+Brand)
    BrandSSpath = BrandPath+"/sub--*.csv"
    filecount = len(gl.glob(BrandSSpath))
    if filecount >= 1:
        for f in range(filecount):
            SSFile = gl.glob(BrandSSpath)[f]
            SSfile = pd.read_csv(SSFile, encoding='mac_roman')
            BrandSS = list(map(str.strip, SSfile.columns))
            SSfile.columns = BrandSS

            if ('tire_width_pd' in BrandSS) == True:
                TireData = (SSfile[(~pd.isnull(SSfile.tire_width_pd)) | (~pd.isnull(SSfile.tire_profile_pd)) 
                                   | (~pd.isnull(SSfile.tire_wheel_dia_pd))]
                            [["internal_sku", "tire_width_pd", "tire_profile_pd", "tire_wheel_dia_pd"]])
                TireData["brand"] = Brand
                TireDataAll = TireDataAll.append(TireData, ignore_index=True)

            if ('wheel_diameter_pd' in BrandSS) == True:
                WheelData = (SSfile[(~pd.isnull(SSfile.wheel_diameter_pd)) | (~pd.isnull(SSfile.wheel_width_pd))
                                    | (~pd.isnull(SSfile.bolt_pattern_pd)) | (~pd.isnull(SSfile.offset_pd)) 
                                    | (~pd.isnull(SSfile.center_bore_pd))]
                            [["internal_sku","wheel_diameter_pd","wheel_width_pd","bolt_pattern_pd","offset_pd","center_bore_pd"]])
                WheelData["brand"] = Brand
                WheelDataAll = WheelDataAll.append(WheelData, ignore_index=True)

                
WheelDataAll.to_csv (r'\\192.168.2.32\Group\Data Team\Abul\3. Final Folder\WheelDataAll.csv', index = None, header=True) 
TireDataAll.to_csv (r'\\192.168.2.32\Group\Data Team\Abul\3. Final Folder\TireDataAll.csv', index = None, header=True) 



In [ ]:
#Identify all Mainsheet with column meta_title that has "Canada" in string in any row

AllBrandFolderPath = "//192.168.2.32/GoogleDrive/Completed Magento Uploads (v 1.0)"
AllBrandFolders = os.listdir(AllBrandFolderPath)

for b in range(len(AllBrandFolders)):
    #print((len(AllBrandFolders)-b))
    Brand = AllBrandFolders[b]
    BrandPath= str(AllBrandFolderPath+"/"+Brand)
    BrandMSpath = BrandPath+"/main--*.csv"
    filecount = len(gl.glob(BrandMSpath))
    if filecount == 1:
        MSFilepath = gl.glob(BrandMSpath)[0]     
        MSFile = pd.read_csv(MSFilepath, encoding='mac_roman')
        BrandMS = list(map(str.strip, MSFile.columns))
        MSFile.columns = BrandMS          
        C = len([i for i in MSFile.meta_title.astype(str).unique() if "Canada" in i])
        L = len([i for i in MSFile.meta_title.astype(str).unique() if "canada" in i])
        t = C+L
        if t != 0:
            print("*",Brand)



In [ ]:
#Compile Category and Image data
AllBrandFolderPath = "//192.168.2.32/GoogleDrive/Completed Magento Uploads (v 1.0)"
AllBrandFolders = os.listdir(AllBrandFolderPath)

CompiledMainSheet = pd.DataFrame(columns = ["categories"])
TargetCol = ["categories", "image1","image2","image3","image4"]

for b in range(len(AllBrandFolders)):
    print((len(AllBrandFolders)-b))
    Brand = AllBrandFolders[b];     print(Brand)
    BrandPath= str(AllBrandFolderPath+"/"+Brand)
    BrandMSpath = BrandPath+"/main--*.csv"
    filecount = len(gl.glob(BrandMSpath))

    if filecount == 1:
        MSFile = gl.glob(BrandMSpath)[0]     
        BrandMS = pd.read_csv(MSFile, encoding='mac_roman')
        BrandMS.columns = map(str.strip, list(BrandMS.columns))
        BrandMS = BrandMS[TargetCol]
        CompiledMainSheet = CompiledMainSheet.append(BrandMS, ignore_index=True)

        
Catdf   = CompiledMainSheet[~CompiledMainSheet["categories"].isna()]
Catdf_1 = Catdf[["categories", "image1"]]
Catdf_2 = Catdf[["categories", "image2"]]
Catdf_3 = Catdf[["categories", "image3"]]
Catdf_4 = Catdf[["categories", "image4"]]

Catdf_1.columns = ["categories", "image"]
Catdf_2.columns = ["categories", "image"]
Catdf_3.columns = ["categories", "image"]
Catdf_4.columns = ["categories", "image"]

Catdf_Pooled = Catdf_1.append(Catdf_2)
Catdf_Pooled = Catdf_Pooled.append(Catdf_3)
Catdf_Pooled = Catdf_Pooled.append(Catdf_4)

#Do not want NA images
Catdf_Pooled = Catdf_Pooled[~Catdf_Pooled["image"].isna()]

#Do not want single image being mapped to multiple unique categories
Catdf_Pooled["Comb"] = Catdf_Pooled["image"]+"~~"+Catdf_Pooled["categories"]
CatComb = pd.DataFrame(list(Catdf_Pooled["Comb"].unique()))
CatComb.columns = ["Comb"]
Catdf_Pooled = pd.DataFrame([re.split("~~", c) for c in CatComb["Comb"]])
Catdf_Pooled.columns = ["image", "categories"]

#Identify images with a single unique Category
ImageReF = pd.DataFrame(Catdf_Pooled.groupby("image").agg("count")["categories"].sort_values().reset_index())
ImageReF = ImageReF[ImageReF["categories"] == 1][["image"]]

#ID Categories corrisponding to images to a single category
CatRef = pd.DataFrame({"categories":Catdf_Pooled[Catdf_Pooled["image"].isin(ImageReF["image"])]["categories"]
                       .dropna().unique().tolist()})

for c in range(len(CatRef)):
    cat = CatRef.loc[c, "categories"]
    BrokenCat = re.split(";", cat)
    count = len(BrokenCat)
    BrokenCat = [b for b in BrokenCat if b.find("Brands/") == -1]
    Dub = 0
    L1_dub = 0

    if len(BrokenCat)>0:

        BrokenLevel = pd.DataFrame([re.split("/", l) for l in BrokenCat])
        L1_len = len(BrokenLevel.iloc[:,0].dropna().unique().tolist())

        if L1_len==1: L1_dub = 1
        if L1_len >=2 and len(BrokenLevel)>2:
            L1_subset = BrokenLevel.groupby(0).agg("count").sort_values(1, ascending = False).reset_index()
            BrokenLevel = BrokenLevel[BrokenLevel.loc[:,0] == L1_subset.iloc[0,0]]
            L1_dub = 1
                                      
        if L1_dub==1:
            if len(BrokenLevel.columns) ==3:
                BrokenLevel.columns = ["L1", "L2", "L3"]

                L1 = BrokenLevel["L1"].unique()
                L2 = BrokenLevel["L2"].unique()
                L3 = BrokenLevel["L3"].unique()
                if (len(L1)==1 and len(L2)==1 and len(L3)==1): levCount = 3
                if (len(L1)==1 and len(L2)==1 and len(L3)!=1): levCount = 2
                if (len(L1)==1 and len(L2)!=1 and len(L3)!=1): levCount = 1
                if (len(L1)==1 and len(L2)!=1 and len(L3)==1): levCount = 1; Dub = 1

            elif len(BrokenLevel.columns) ==2:
                BrokenLevel.columns = ["L1", "L2"]
                L1 = BrokenLevel["L1"].unique()
                L2 = BrokenLevel["L2"].unique()
                if (len(L1)==1 and len(L2)==1 ): levCount = 2
                if (len(L1)==1 and len(L2)!=1 ): levCount = 1

            elif len(BrokenLevel.columns) ==1:
                BrokenLevel.columns = ["L1"]
                L1 = BrokenLevel["L1"].unique()
                if (len(L1)==1 ): levCount = 1

            if levCount == 3:
                CatRef.loc[c, "count"] = count
                CatRef.loc[c, "levCount"] = levCount
                CatRef.loc[c, "L1"] = L1
                CatRef.loc[c, "L3"] = L3

                if Dub == 0: CatRef.loc[c, "L2"] = L2
                if Dub == 1: CatRef.loc[c, "L2"] = sorted(L2,key=len)[0]

            elif levCount == 2:
                CatRef.loc[c, "count"] = count
                CatRef.loc[c, "levCount"] = levCount
                CatRef.loc[c, "L1"] = L1
                CatRef.loc[c, "L2"] = L2
            elif levCount == 1:
                CatRef.loc[c, "count"] = count
                CatRef.loc[c, "levCount"] = levCount
                CatRef.loc[c, "L1"] = L1

Catdf_Final = Catdf_Pooled.merge(CatRef, on = "categories", how = "outer")
Catdf_Final = Catdf_Final[~Catdf_Final["L1"].isna()]

#Test if there is one-to-one image/category combos
len((Catdf_Final["image"]+"~~"+Catdf_Final["categories"]).unique())
len(Catdf_Final["image"])

Catdf_Final.to_csv(r'\\192.168.2.32\Group\Data Team\Abul\Catdf_Final.csv', index = None, header=True)


In [ ]:
#Zero-sale skus data compile

AllBrandFolderPath = "//192.168.2.32/GoogleDrive/Completed Magento Uploads (v 1.0)"
AllBrandFolders = os.listdir(AllBrandFolderPath)

#NonSoldSkus=pd.read_csv("//192.168.2.32/Group/Data Team/Brand_Update_Location/5_R_Brand_Reference_Files/NonSoldSkus.csv", encoding='utf-8')
#NonSoldSkus = list(NonSoldSkus["internal_sku"])

CompiledMainSheet = pd.DataFrame(columns = ["categories"])
TargetCol = ["internal_sku", "ca_cost","na_ca_shipping","part_type_filter","attribute_set"]
#TargetCol = ["categories", "image1","image2","image3","image4"]
#TargetCol = ["internal_sku", "sku", "ca_price", "ca_retail_price", "ca_cost", "ca_jobber_price", "na_ca_shipping", "na_supplier"]

for b in range(len(AllBrandFolders)):
    print((len(AllBrandFolders)-b))
    Brand = AllBrandFolders[b];     print(Brand)
    BrandPath= str(AllBrandFolderPath+"/"+Brand)
    BrandMSpath = BrandPath+"/main--*.csv"
    filecount = len(gl.glob(BrandMSpath))

    if filecount == 1:
        MSFile = gl.glob(BrandMSpath)[0]     
        BrandMS = pd.read_csv(MSFile, encoding='mac_roman')
        BrandMS.columns = map(str.strip, list(BrandMS.columns))
        BrandMS = BrandMS[(BrandMS["type"] == "simple") & (BrandMS["delete"] == "N")]
        BrandMS = BrandMS[TargetCol]

        #BrandMS = BrandMS[BrandMS["internal_sku"].isin(NonSoldSkus)]
        CompiledMainSheet = CompiledMainSheet.append(BrandMS, ignore_index=True)

            
(CompiledMainSheet.to_csv(r'\\192.168.2.32\Group\Data Team\Abul\CompiledMainSheet.csv', index = None, header=True))



In [ ]:

PriceAnalysis = CompiledMainSheet

PriceAnalysis = (PriceAnalysis[~(PriceAnalysis["na_ca_shipping"].isna()) &
                                     ~(PriceAnalysis["ca_cost"].isna()) &
                                    (PriceAnalysis["ca_cost"] != "TBA")])

PriceAnalysis["price"] = (PriceAnalysis["na_ca_shipping"].astype(float) + PriceAnalysis["ca_cost"].astype(float)) 
PriceAnalysis["ShipCostRatio"] = round(PriceAnalysis["na_ca_shipping"].astype(float) / PriceAnalysis["price"], 1)

PriceAnalysis[PriceAnalysis["ShipCostRatio"] >= .5]
PriceAnalysis.groupby("ShipCostRatio").agg("count").reset_index()[["ShipCostRatio", "na_ca_shipping"]]

A = PriceAnalysis.groupby("part_type_filter")["ShipCostRatio"].agg("mean").reset_index()#[["ShipCostRatio", "na_ca_shipping"]]

(A.to_csv(r'\\192.168.2.32\Group\Data Team\Abul\4. Temp Folder\PTshippingPercent.csv', index = None, header=True))

(CompiledMainSheet.to_csv(r'\\192.168.2.32\Group\Data Team\Abul\CompiledMainSheet.csv', index = None, header=True))


In [ ]:
AllBrandFolderPath = "//192.168.2.32/GoogleDrive/Completed Magento Uploads (v 1.0)"
AllBrandFolders = os.listdir(AllBrandFolderPath)

Ref = pd.DataFrame({"brands":AllBrandFolders, "MS":0, "SS":0})

for b in range(len(Ref)):
    Brand = Ref.loc[b, "brands"]
    BrandPath= str(AllBrandFolderPath+"/"+Brand)
    BrandMSpath = BrandPath+"/*main--*.csv"
    BrandSSpath = BrandPath+"/*sub*.csv"

    Ref.loc[b, "MS"] = len(gl.glob(BrandMSpath))
    Ref.loc[b, "SS"] = len(gl.glob(BrandSSpath))
    
(Ref.to_csv(r'\\192.168.2.32\Group\Data Team\Abul\Ref.csv', index = None, header=True))


    print("CSV Count:",filecountMS, filecountSS, " Brand:", Brand)

In [ ]:
#Zero-sale skus data compile

AllBrandFolderPath = "//192.168.2.32/GoogleDrive/Completed Magento Uploads (v 1.0)"
AllBrandFolders = os.listdir(AllBrandFolderPath)


TargetPT =[ 	
            "Floor Mats", 	
            "Brake Kits", 	
            "Performance Chips & Programmers", 	
            "Shocks & Struts"]#, 	
            #"Coil-Overs", 	
            #"Suspension Lift Kits", 	
            #"Wheels", 	
            #"Headlight Assemblies", 	
            #"Nerf Bars & Step Bars"]

for b in range(192, len(AllBrandFolders)):
    print((len(AllBrandFolders)-b))
    Brand = AllBrandFolders[b];     print(Brand)
    BrandPath= str(AllBrandFolderPath+"/"+Brand)
    BrandMSpath = BrandPath+"/main--*.csv"
    BrandSSpath = BrandPath+"/sub--*.csv"
    filecount = len(gl.glob(BrandMSpath))
    filecountSS = len(gl.glob(BrandSSpath))

    PTFolderPath = "//192.168.2.32/GoogleDrive/PT_mainsheet"
    AllPTfiles = os.listdir(PTFolderPath)

    if filecount == 1:
        MSFile = gl.glob(BrandMSpath)[0]
        BrandMS = pd.read_csv(MSFile, encoding='mac_roman')
        BrandMS.columns = [c.strip() for c in BrandMS.columns]

        brandPT = BrandMS["part_type_filter"].dropna().unique().tolist()
        brandPT = [c.strip() for c in brandPT]
        brandPT = [p for p in brandPT if p in TargetPT]

        if len(brandPT) != 0:
            for p in range(len(brandPT)):
                pt = brandPT[p]; print(pt)
                pt_new = re.sub("-", "_", pt)
                pt_new = re.sub("&", "and", pt_new)
                pt_new = re.sub(";", "_or_", pt_new)
                pt_new = re.sub("/", "_", pt_new)
                pt_new = re.sub(" ", "_", pt_new)
                pt_new = re.sub("__", "_", pt_new)
                pt_new = re.sub("__", "_", pt_new)

                pt_file = "main--"+pt_new+".csv"
                pt_path = PTFolderPath+"/"+pt_file
                pt_fileSS = "sub--"+pt_new+".csv"
                pt_pathSS = PTFolderPath+"/"+pt_fileSS

                #Mainsheet Sku Pull
                PtSubset = BrandMS[BrandMS["part_type_filter"] == pt]
                PtSubset = PtSubset.dropna(axis=1,how='all')
                PTintSku = PtSubset["internal_sku"].dropna().unique().tolist()

                #Series Pull
                Pt_serieslist = list(PtSubset["series_parent"].unique())
                SeriesSubset = BrandMS[(BrandMS["type"]== "series") &
                                       (BrandMS["internal_sku"].isin(Pt_serieslist))]
                SeriesSubset = SeriesSubset.dropna(axis=1,how='all')

                #Append sku with series
                finalsubset = PtSubset.append(SeriesSubset,sort=False)

                if (pt_file in AllPTfiles) == True:

                    pt_fileMS_test = PTFolderPath+"/"+"main--"+pt_new+"*.csv"
                    num = str(len(gl.glob(pt_fileMS_test)))
                    LatestMSfile = gl.glob(pt_fileMS_test)[-1]
                    MSfileSize = os.path.getsize(LatestMSfile)*0.001

                    if MSfileSize <= 100000:
                        Old_PTfile=pd.read_csv(LatestMSfile, encoding='utf-8')
                        New_PTfile = Old_PTfile.append(finalsubset, sort=False)
                        New_PTfile.to_csv(LatestMSfile, index = None, header=True,  encoding='utf-8')

                    else:
                        newMSPath = PTFolderPath+"/"+"main--"+pt_new+"_"+num+".csv"
                        finalsubset.to_csv(newMSPath, index = None, header=True,  encoding='utf-8')

                else:
                    finalsubset.to_csv(pt_path, index = None, header=True,  encoding='utf-8')


               #Pull out Subsheet 
                if filecountSS >= 1:              
                    SSFile = gl.glob(BrandSSpath)   
                    BrandSS = pd.DataFrame()
                    
                    for s in range(len(SSFile)):
                        BrandSS_single = pd.read_csv(SSFile[s], encoding='mac_roman')
                        BrandSS_single.columns = [c.strip() for c in BrandSS_single.columns]
                        BrandSS = BrandSS.append(BrandSS_single, sort=False)

                    #Subset Subsheet
                    PTSubSheet = BrandSS[BrandSS["internal_sku"].isin(PTintSku)]
                    PTSubSheet = PTSubSheet.dropna(axis=1,how='all')
                                                                            
                    if (pt_fileSS in AllPTfiles) == True:

                        pt_fileSS_test = PTFolderPath+"/"+"sub--"+pt_new+"*.csv"
                        num = str(len(gl.glob(pt_fileSS_test)))
                        LatestSSfile = gl.glob(pt_fileSS_test)[-1]
                        SSfileSize = os.path.getsize(LatestSSfile)*0.001

                        if SSfileSize <= 100000:
                            Old_SSfile=pd.read_csv(LatestSSfile, encoding='utf-8')
                            New_SSfile = Old_SSfile.append(PTSubSheet, sort=False)
                            New_SSfile.to_csv(LatestSSfile, index = None, header=True,  encoding='utf-8')

                        else:
                            newSSPath = PTFolderPath+"/"+"sub--"+pt_new+"_"+num+".csv"
                            PTSubSheet.to_csv(newSSPath, index = None, header=True,  encoding='utf-8')

                    else:
                        PTSubSheet.to_csv(pt_pathSS, index = None, header=True,  encoding='utf-8')



In [ ]:
#Load MasterDS_R file
MasterDS=pd.read_csv("//192.168.2.32/Group/Data Team/Recommender_System_Location/1_Reference_Files/MasterDS.csv", encoding='utf-8')

list(MasterDS.columns)

ShippingData_R = MasterDS[["Order Date", "Product Cost (CAD)", "Shipping Cost (CAD)", "Net Retail Price (CAD)", "Province", "part_type_filter"]]
ShippingData_R["Shipping Cost (CAD)"] = [float(  (str(s).replace("$", "")).replace(",", "")  ) for s in ShippingData_R["Shipping Cost (CAD)"] ]
ShippingData_R["Net Retail Price (CAD)"] = [float(  (str(s).replace("$", "")).replace(",", "")  ) for s in ShippingData_R["Net Retail Price (CAD)"] ]

ShippingData  = ShippingData_R[~(ShippingData_R["Product Cost (CAD)"].isna()) &
                               ~(ShippingData_R["Shipping Cost (CAD)"].isna()) &
                               ~(ShippingData_R["Net Retail Price (CAD)"].isna()) &

                               ~(ShippingData_R["part_type_filter"].isna()) &
                               ~(ShippingData_R["Product Cost (CAD)"] == 0) &
                               ~(ShippingData_R["Shipping Cost (CAD)"] == 0) &
                               ~(ShippingData_R["Net Retail Price (CAD)"] == 0) 
                              ]


ShippingData["price"] = (ShippingData["Shipping Cost (CAD)"].astype(float) + ShippingData["Product Cost (CAD)"].astype(float)) 
ShippingData["ShipCostRatio"] = round(ShippingData["Shipping Cost (CAD)"].astype(float) / ShippingData["price"], 1)

ShippingData['Order_Date'] = pd.to_datetime(ShippingData['Order Date'], format= "%d-%b-%y")
ShippingData['OD_Year'] = pd.to_numeric(ShippingData['Order_Date'].dt.strftime('%Y'))

A = ShippingData.groupby("part_type_filter")["ShipCostRatio"].agg("mean").reset_index()#[["ShipCostRatio", "na_ca_shipping"]]

B = ShippingData.groupby(["part_type_filter", "Province"]).agg("count").reset_index()[["part_type_filter", "Province", "Order Date"]]
B = B.pivot(index='part_type_filter', columns='Province', values='Order Date').reset_index()

C = ShippingData.groupby(["part_type_filter", "OD_Year"])["Net Retail Price (CAD)"].agg("sum").reset_index()[["part_type_filter", "OD_Year", "Net Retail Price (CAD)"]]
C = C.pivot(index='part_type_filter', columns='OD_Year', values='Net Retail Price (CAD)').reset_index()
C.columns = ["part_type_filter", "2015_Rev", "2016_Rev", "2017_Rev", "2018_Rev", "2019_Rev"]

Z = A.merge(B, on = "part_type_filter", how = "outer")
Z = Z.merge(C, on = "part_type_filter", how = "outer")


(Z.to_csv(r'\\192.168.2.32\Group\Data Team\Abul\4. Temp Folder\PTshippingPerProv.csv', index = None, header=True))

